<a href="https://colab.research.google.com/github/cassioHilario/DomHelder/blob/main/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
import torch
import spacy
import cupy as cp
import random

In [ ]:
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.matcher import Matcher


In [ ]:
spacy.prefer_gpu()
nlp = spacy.load("pt_core_news_lg")

In [ ]:
pasta = '../Base/data/'
pastan = '../Base/data.V2/'

# Nome do arquivo de saída onde as três últimas linhas serão salvas
arquivo_sem_erro = '../Base/data.V2/amostra_base_sem erros.txt'

In [ ]:
# Função para processar e escrever uma linha com quebra de linha após a pontuação
def processar_linha(linha):
    # Remove espaços em branco antes da pontuação
    linha = re.sub(r'\s+([.!?;,:])', r'\1', linha)
    # Adiciona uma quebra de linha após a pontuação
    linha = re.sub(r'([.!?;,:])', r'\1\n', linha)
    return linha

In [ ]:
arquivos = os.listdir(pasta)
total_arquivos = int(len(os.listdir(pasta))*0.1)
arquivos_selecionados = random.sample(arquivos, total_arquivos)
for nome_arquivo in arquivos_selecionados:
        # Verifique se o arquivo é um arquivo TXT
        if nome_arquivo.endswith('.txt'):
            arquivo_path = os.path.join(pasta, nome_arquivo)
            arquivo_saida_path = os.path.join(pastan, nome_arquivo)
            try:
                conteudo = ''
                # Abra o arquivo e leia as três últimas linhas
                with open(arquivo_path, 'r', encoding='utf-8') as arquivo:
                    conteudo = arquivo.read()
                with open(arquivo_saida_path, 'w', encoding='utf-8') as arquivo:
                    conteudo = processar_linha(conteudo)
                    arquivo.write(conteudo.strip())
            except Exception as e:
                print(f"Erro ao processar {arquivo_path}: {str(e)}")

In [ ]:
class Noun:
    def __init__(self, gender, number):
        self._gender = gender
        self._number = number

    @property
    def gender(self):
        return self._gender

    @gender.setter
    def gender(self, value):
        self._gender = value

    @property
    def number(self):
        return self._number

    @number.setter
    def number(self, value):
        self._number = value


In [ ]:

def get_noun(doc):
    for token in doc:
        if token.pos_ == 'NOUN':
            return Noun(token.morph.get('Gender'), token.morph.get('Number'))


In [ ]:
def isCorrect(doc, noun):
    for token in doc:
        if token.pos_ == 'DET' or token.pos_ == 'PRON' or token.pos_ == 'ADJ':
            if token.morph.get('Gender') != noun.gender or token.morph.get('Number') != noun.number:
                return False
        elif token.pos_ == 'AUX' or token.pos_ == 'VERB':
            if token.morph.get('Number') != noun.number:
                return False
    return True

In [ ]:
# Define the patterns
patterns = [
    [{"POS": "DET"}, {"POS": "NOUN"}, {"POS": "VERB"}, {"POS": "AUX"}, {"POS": "ADJ"}],
    [{"POS": "DET"}, {"POS": "NOUN"}, {"POS": "AUX"}, {"POS": "VERB"}, {"POS": "ADJ"}],
    [{"POS": "DET"}, {"POS": "NOUN"}, {"POS": "VERB"}, {"POS": "ADJ"}],
    [{"POS": "DET"}, {"POS": "NOUN"}, {"POS": "AUX"}, {"POS": "ADJ"}],
    [{"POS": "DET"}, {"POS": "NOUN"}, {"POS": "ADJ"}]
]

# Initialize the matcher with the patterns
matcher = Matcher(nlp.vocab)
matcher.add("SENTENCE_STRUCTURE", patterns)


In [ ]:
def posses(doc):
    posses = []
    for token in doc:
        if token.pos == "ADJ" or token.pos == "NOUN" or token.pos == "DET" or token.pos == "PRON":
            if token.morph not in posses:
                posses.append(str(token.morph))

In [ ]:
def pick_and_remove(lst):
    index = random.randint(0, len(lst) - 1)
    picked_string = lst[index]
    lst.remove(picked_string)
    return picked_string

In [ ]:
def changeGender(token):
    if token.morph.get('Gender') == 'Masc':
        if token.text.endswith('o'):
            return token.text[:-1] + 'a'
        elif token.text.endswith('or'):
            return token.text[:-2] + 'ora'
        elif token.text.endswith('ão'):
            return token.text[:-2] + 'ã'
        elif token.text.endswith('ês'):
            return token.text[:-2] + 'esa'
        elif token.text.endswith('r'):
            return token.text + 'a'
        elif token.text.endswith('l'):
            return token.text + 'a'
        elif token.text.endswith('z'):
            return token.text + 'a'
        elif token.text.endswith('e'):
            return token.text + 'a'
    elif token.morph.get('Gender') == 'Fem':
        if token.text.endswith('a'):
            return token.text[:-1] + 'o'
        elif token.text.endswith('ora'):
            return token.text[:-1]
        elif token.text.endswith('ã'):
            return token.text + 'o'
        elif token.text.endswith('esa'):
            return token.text[:-1] + 'o'
        elif token.text.endswith('r'):
            return token.text[:-1] + 'or'
        elif token.text.endswith('l'):
            return token.text[:-1] + 'or'
        elif token.text.endswith('z'):
            return token.text[:-1] + 'or'
        elif token.text.endswith('e'):
            return token.text[:-1] + 'or'
    return token.text


In [ ]:
def changeNumber(token):
    if token.morph.get('Number') == 'Sing':
        if token.text.endswith('s'):
            return token.text[:-1]
        elif token.text.endswith('ão'):
            return token.text[:-2] + 'ões'
        elif token.text.endswith('ês'):
            return token.text[:-2] + 'eses'
        elif token.text.endswith('l'):
            return token.text + 's'
        elif token.text.endswith('r'):
            return token.text + 'es'
        elif token.text.endswith('z'):
            return token.text + 'es'
        elif token.text.endswith('m'):
            return token.text[:-1] + 'ns'
        else:
            return token.text + 's'
    elif token.morph.get('Number') == 'Plur':
        if token.text.endswith('s'):
            return token.text + 'es'
        elif token.text.endswith('ão'):
            return token.text[:-2] + 'ões'
        elif token.text.endswith('ês'):
            return token.text[:-2] + 'eses'
        elif token.text.endswith('l'):
            return token.text + 'es'
        elif token.text.endswith('r'):
            return token.text + 'es'
        elif token.text.endswith('z'):
            return token.text + 'es'
        elif token.text.endswith('m'):
            return token.text[:-1] + 'ns'
        else:
            return token.text + 's'
    return token.text



In [ ]:
def changeWord(token, i):
    if token.pos_ == "NOUN" or token.pos_ == "PRON" or token.pos_ == "DET" or token.pos_ == "ADJ":
        if i % 2 == 0:
            return changeGender(token)
        else:
            return changeNumber(token)
    return token.text

In [ ]:
def insertErro(doc):
    posses = posses(doc)
    num_erros = random.randint(1, len(posses))
    words = []
    for i in range(num_erros):
        pos = pick_and_remove(posses)
        for token in doc:
            word = token.text
            if str(token.pos_) == pos:
                word = changeWord(token)
            if len(words) <= token.i:
                words.append(word, i)
            else:
                words[token.i] = word
    return Doc(doc.vocab, words=words)


In [ ]:
def doc_to_string(doc):
    return doc.text


In [ ]:
def isMatch(doc):
    matches = matcher(doc)
    return len(matches) > 0

In [ ]:



# Contador para acompanhar o número de arquivos processados
contador_arquivos = 0

# Abrir o arquivo de saída para escrita
with open(arquivo_sem_erro, 'w', encoding='utf-8') as f:

    linhas_corretas = []
    # Loop através dos arquivos na pasta
    for nome_arquivo in os.listdir(pastan):
        # Verifique se o arquivo é um arquivo TXT
        if nome_arquivo.endswith('.txt'):
            arquivo_path = os.path.join(pastan, nome_arquivo)
            try:
                # Abra o arquivo e leia as três últimas linhas
                with open(arquivo_path, 'r', encoding='utf-8') as arquivo:
                    contador_linhas = 0
                    linhas = []
                    linhas = arquivo.readlines()
                    linhas.reverse()
                    # Processa e escreve essas linhas no arquivo de saída
                    for linha in linhas:
                        # verifica se a linha é vazia
                        if linha.strip() != '':
                            # transforma em doc
                            doc = nlp(linha)
                            if isMatch(doc):
                                # captura o genero e numero do substantivo
                                noun = get_noun(doc)
                                if isCorrect(doc, noun):
                                    linhas_corretas.append(linha.strip())
                                    #arquivo_saida.write('\n')
                                    #arquivo_saida.write(doc_to_string(insertErro(doc)))
                                    contador_linhas += 1
                                    if contador_linhas == 3:
                                        break
                    contador_arquivos += 1
                    # Verifique se atingimos o limite de 300.000 arquivos
            except Exception as e:
                print(f"Erro ao processar {arquivo_path}: {str(e)}")
    f.write("".join(linha+"\n" for linha in linhas_corretas))

print(f"{contador_arquivos} arquivos processados e as três últimas linhas foram salvas em {f.name}")